# EcoHome Energy Advisor - Database Setup

In this notebook, you'll set up the database for the EcoHome Energy Advisor. The database will store:
- Energy usage data (consumption, device types, costs)
- Solar generation data (production, weather conditions)

## Learning Objectives
- Create SQLite database with proper schema
- Populate database with sample data
- Query data for analysis
- Understand database design for energy management


## 1. Import Required Libraries


In [1]:
# Import the necessary libraries
from datetime import datetime, timedelta
import random

from models.energy import DatabaseManager

## 2. Initialize Database Manager


In [2]:
# Create a DatabaseManager instance
# Initialize it with the path "data/energy_data.db"
db_manager = DatabaseManager("data/energy_data.db")


## 3. Create Database Tables


In [3]:
# Create the database tables
# Use the create_tables() method from your DatabaseManager
db_manager.create_tables()


Database tables created at data/energy_data.db


## 4. Generate Sample Energy Usage Data


In [4]:
# Generate sample energy usage data for the past 30 days
# Include different device types: EV, HVAC, appliances
# Use realistic consumption patterns (higher during peak hours)

# Device types and their typical consumption patterns
device_types = {
    'EV': {'base_kwh': 10, 'variation': 5, 'peak_hours': [18, 19, 20, 21]},
    'HVAC': {'base_kwh': 2, 'variation': 1, 'peak_hours': [12, 13, 14, 15, 16, 17]},
    'appliance': {'base_kwh': 1.5, 'variation': 0.5, 'peak_hours': [19, 20, 21, 22]}
}

# Generate data for the past 30 days
start_date = datetime.now() - timedelta(days=30)
records_created = 0

for day in range(30):
    current_date = start_date + timedelta(days=day)
    
    for hour in range(24):
        timestamp = current_date.replace(hour=hour, minute=0, second=0, microsecond=0)
        
        # Generate usage for each device type
        for device_type, config in device_types.items():
            # Base consumption with variation
            base_consumption = config['base_kwh']
            variation = random.uniform(-config['variation'], config['variation'])
            
            # Peak hour multiplier
            peak_multiplier = 1.5 if hour in config['peak_hours'] else 0.8
            
            consumption = max(0, (base_consumption + variation) * peak_multiplier)
            
            # Calculate cost (simplified pricing)
            price_per_kwh = 0.15 if hour in config['peak_hours'] else 0.10
            cost = consumption * price_per_kwh
            
            # Device names
            device_names = {
                'EV': 'Tesla Model 3',
                'HVAC': 'Main AC Unit',
                'appliance': random.choice(['Dishwasher', 'Washing Machine', 'Dryer'])
            }
            
            # Add record to database
            db_manager.add_usage_record(
                timestamp=timestamp,
                consumption_kwh=consumption,
                device_type=device_type,
                device_name=device_names[device_type],
                cost_usd=cost
            )
            records_created += 1

print(f"Created {records_created} energy usage records")


Created 2160 energy usage records


## 5. Generate Sample Solar Generation Data


In [5]:
# Generate sample solar generation data
# Consider weather patterns, time of day, and seasonal variations

# Weather conditions and their impact on solar generation
weather_conditions = {
    'sunny': {'multiplier': 1.0, 'probability': 0.4},
    'partly_cloudy': {'multiplier': 0.6, 'probability': 0.3},
    'cloudy': {'multiplier': 0.3, 'probability': 0.2},
    'rainy': {'multiplier': 0.1, 'probability': 0.1}
}

# Generate solar generation data for the same period
start_date = datetime.now() - timedelta(days=30)
generation_records = 0

for day in range(30):
    current_date = start_date + timedelta(days=day)
    
    # Select weather condition for the day
    weather_choice = random.choices(
        list(weather_conditions.keys()),
        weights=[weather_conditions[w]['probability'] for w in weather_conditions.keys()]
    )[0]
    weather_multiplier = weather_conditions[weather_choice]['multiplier']
    
    for hour in range(24):
        timestamp = current_date.replace(hour=hour, minute=0, second=0, microsecond=0)
        
        # Solar generation only during daylight hours (6 AM to 6 PM)
        if 6 <= hour <= 18:
            # Base generation curve (peak at noon)
            hour_factor = 1 - abs(hour - 12) / 6  # Peak at hour 12
            base_generation = 5.0 * hour_factor  # Max 5 kWh at peak
            
            # Apply weather multiplier
            generation = base_generation * weather_multiplier
            
            # Add some random variation
            generation *= random.uniform(0.8, 1.2)
            generation = max(0, generation)
            
            # Calculate temperature (affects solar efficiency)
            base_temp = 20 + random.uniform(-5, 5)
            temp_factor = 1.0 if 15 <= base_temp <= 35 else 0.9
            
            # Solar irradiance calculation
            irradiance = 800 * hour_factor * weather_multiplier if generation > 0 else 0
            
            # Add record to database
            db_manager.add_generation_record(
                timestamp=timestamp,
                generation_kwh=generation,
                weather_condition=weather_choice,
                temperature_c=base_temp * temp_factor,
                solar_irradiance=irradiance
            )
            generation_records += 1

print(f"Created {generation_records} solar generation records")


Created 390 solar generation records


## 6. Query and Analyze Data


In [6]:
# Query the data to verify it was inserted correctly
# Show total consumption by device type
# Show daily solar generation patterns
# Calculate some basic statistics

# Get recent data for analysis
recent_usage = db_manager.get_recent_usage(24)  # Last 24 hours
recent_generation = db_manager.get_recent_generation(24)

print("=== Energy Usage Analysis ===")
print(f"Total records in last 24 hours: {len(recent_usage)}")

# Group by device type
device_consumption = {}
for record in recent_usage:
    device = record.device_type or 'unknown'
    if device not in device_consumption:
        device_consumption[device] = {'kwh': 0, 'cost': 0, 'records': 0}
    device_consumption[device]['kwh'] += record.consumption_kwh
    device_consumption[device]['cost'] += record.cost_usd or 0
    device_consumption[device]['records'] += 1

print("\nConsumption by device type:")
for device, data in device_consumption.items():
    print(f"  {device}: {data['kwh']:.2f} kWh, ${data['cost']:.2f}, {data['records']} records")

print(f"\n=== Solar Generation Analysis ===")
print(f"Total generation records in last 24 hours: {len(recent_generation)}")

total_generation = sum(r.generation_kwh for r in recent_generation)
print(f"Total generation: {total_generation:.2f} kWh")

# Weather breakdown
weather_breakdown = {}
for record in recent_generation:
    weather = record.weather_condition or 'unknown'
    if weather not in weather_breakdown:
        weather_breakdown[weather] = {'kwh': 0, 'records': 0}
    weather_breakdown[weather]['kwh'] += record.generation_kwh
    weather_breakdown[weather]['records'] += 1

print("\nGeneration by weather condition:")
for weather, data in weather_breakdown.items():
    print(f"  {weather}: {data['kwh']:.2f} kWh, {data['records']} records")


=== Energy Usage Analysis ===
Total records in last 24 hours: 33

Consumption by device type:
  EV: 121.01 kWh, $15.38, 11 records
  HVAC: 25.86 kWh, $3.43, 11 records
  appliance: 16.99 kWh, $2.10, 11 records

=== Solar Generation Analysis ===
Total generation records in last 24 hours: 6
Total generation: 7.53 kWh

Generation by weather condition:
  partly_cloudy: 7.53 kWh, 6 records


## 7. Test Database Tools


In [7]:
# Test the database query functions from tools.py
# Import and test: query_energy_usage, query_solar_generation, get_recent_energy_summary

from tools import query_energy_usage, query_solar_generation, get_recent_energy_summary

# Test querying data for the last 7 days
end_date = datetime.now().strftime("%Y-%m-%d")
start_date = (datetime.now() - timedelta(days=7)).strftime("%Y-%m-%d")

print("=== Testing Database Tools ===")
print(f"Querying data from {start_date} to {end_date}")

# Test energy usage query
usage_data = query_energy_usage.invoke(
    input={
        "start_date": start_date, 
        "end_date": end_date,
    }
)
print(f"\nEnergy Usage Query Results:")
print(f"  Total records: {usage_data['total_records']}")
print(f"  Total consumption: {usage_data['total_consumption_kwh']} kWh")
print(f"  Total cost: ${usage_data['total_cost_usd']}")

# Test solar generation query
generation_data = query_solar_generation.invoke(
    input={
        "start_date": start_date, 
        "end_date": end_date,
    }
)
print(f"\nSolar Generation Query Results:")
print(f"  Total records: {generation_data['total_records']}")
print(f"  Total generation: {generation_data['total_generation_kwh']} kWh")
print(f"  Average daily: {generation_data['average_daily_generation']} kWh")

# Test recent energy summary
summary = get_recent_energy_summary.invoke(
    input={
        "hours": 24
    }
)
print(f"\nRecent Energy Summary:")
print(f"  Usage: {summary['usage']['total_consumption_kwh']} kWh, ${summary['usage']['total_cost_usd']}")
print(f"  Generation: {summary['generation']['total_generation_kwh']} kWh")
print(f"  Weather: {summary['generation']['average_weather']}")


=== Testing Database Tools ===
Querying data from 2026-02-09 to 2026-02-16

Energy Usage Query Results:
  Total records: 504
  Total consumption: 2052.18 kWh
  Total cost: $234.99

Solar Generation Query Results:
  Total records: 91
  Total generation: 158.06 kWh
  Average daily: 19.76 kWh

Recent Energy Summary:
  Usage: 163.86 kWh, $20.91
  Generation: 7.53 kWh
  Weather: sunny
